In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def mean_shift_step(X, y0, epsilon, max_iter=100):
    """
    Mean Shift 단일 단계 구현
    """
    t = 0
    while True:
        # 반경 epsilon 내의 포인트 찾기
        distances = np.linalg.norm(X - y0, axis=1)
        points_in_radius = X[distances <= epsilon]

        if len(points_in_radius) == 0:
            break

        # 평균 계산
        y_next = np.mean(points_in_radius, axis=0)

        # 수렴 조건
        if np.linalg.norm(y_next - y0) <= epsilon:
            break

        y0 = y_next
        t += 1

        if t >= max_iter:
            break

    return y0

def apply_mean_shift_to_image(image_path, epsilon=10.0, min_area=100):
    """
    Mean Shift를 적용하여 이미지 세그먼테이션 수행
    """
    # 이미지 불러오기
    image = cv2.imread(image_path)
    if image is None:
        print(f"이미지를 불러올 수 없습니다. 경로를 확인하세요: {image_path}")
        return None, None

    # RGB에서 Luv로 변환
    image_luv = cv2.cvtColor(image, cv2.COLOR_BGR2LUV)
    height, width, channels = image_luv.shape
    X = image_luv.reshape(-1, 3).astype(float)  # 픽셀 데이터를 2D 배열로 변환

    # 각 픽셀에 대해 Mean Shift 적용
    n = X.shape[0]
    V = np.zeros((n, 3))  # 클러스터 중심 저장
    for i in range(n):
        V[i] = mean_shift_step(X, X[i], epsilon)

    # 클러스터 레이블링 (간단한 k-means 스타일 할당)
    labels = np.argmin(np.linalg.norm(V[:, np.newaxis] - V, axis=2), axis=0)

    # 클러스터 병합 (min_area 기준)
    unique_labels = np.unique(labels)
    cluster_sizes = np.bincount(labels)
    final_labels = labels.copy()

    for label in unique_labels:
        if cluster_sizes[label] < min_area:
            # 작은 클러스터를 가장 가까운 큰 클러스터에 병합
            mask = (labels == label)
            distances_to_other_centers = np.linalg.norm(V[mask] - V[labels != label], axis=1)
            if len(distances_to_other_centers) > 0:
                nearest_large_label = np.argmin(distances_to_other_centers)
                final_labels[mask] = labels[labels != label][nearest_large_label]

    # 세그먼테이션 이미지 생성
    segmented_image = V[final_labels].reshape(height, width, 3).astype(np.uint8)
    segmented_rgb = cv2.cvtColor(segmented_image, cv2.COLOR_LUV2RGB)

    return image, segmented_rgb

# 시각화
if __name__ == "__main__":
    # 이미지 파일 경로 설정
    image_path = "./image/lena.bmp"  # 실제 이미지 경로로 대체

    # Mean Shift 적용
    original_image, segmented_image = apply_mean_shift_to_image(image_path, epsilon=10.0, min_area=100)

    if original_image is not None and segmented_image is not None:
        # 원본 이미지와 세그먼테이션 결과 비교
        plt.figure(figsize=(12, 6))

        plt.subplot(121)
        plt.imshow(cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB))
        plt.title("Original Image")
        plt.axis('off')

        plt.subplot(122)
        plt.imshow(segmented_image)
        plt.title("Segmented Image (Mean Shift)")
        plt.axis('off')

        plt.show()


KeyboardInterrupt: 